# Verb <-> Categ

In [ ]:
# gerar_visualizacao_heterogenea.py
#
# Versão adaptada para visualizar uma rede heterogênea com
# nós de 'verbetes' e 'categorias', cada um com um estilo visual distinto.

import json
import os
from datetime import datetime
import math

def gerar_visualizacao_heterogenea():

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    input_dir = 'dados'
    # ATUALIZAÇÃO: Lendo o novo arquivo com dados heterogêneos
    input_filename = 'dados_rede_heterogenea.json'
    output_dir = 'Visualizações'
    
    input_path = os.path.join(input_dir, input_filename)

    if not os.path.exists(input_path):
        print(f"ERRO: O arquivo de dados '{input_path}' não foi encontrado.")
        return

    # --- 2. CARREGAMENTO DOS DADOS HETEROGÊNEOS ---
    print(f"Carregando dados da rede heterogênea de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    # A lista de nós agora contém tanto verbetes quanto categorias
    all_nodes_data = dados['nodes']
    
    # Cria dicionários para busca rápida
    id_to_node = {str(node['id']): node for node in all_nodes_data}
    titulo_to_id = {node['titulo']: str(node['id']) for node in all_nodes_data if node.get('type') == 'verbete'}

    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados de nós, arestas e filtros para o Cytoscape...")
    
    nodes_for_cy = []
    verbetes_data = {}
    all_categories = set()
    community_map = {}
    nodes_to_render = []

    # ATUALIZAÇÃO: Primeiro, filtramos os nós que serão renderizados
    nodes_to_render = []
    for node in all_nodes_data:
        # Condição para incluir o nó:
        # 1. Se for uma categoria, inclua.
        # 2. Se for um verbete, inclua APENAS se community_id for diferente de -1.
        if node.get('type') == 'category' or (node.get('type') == 'verbete' and node.get('community_id', -1) != -1):
            nodes_to_render.append(node)

    print(f"Renderizando {len(nodes_to_render)} de {len(all_nodes_data)} nós (filtrando verbetes isolados).")

    for node in nodes_to_render:
        node_id = str(node['id'])
        
        # ATUALIZAÇÃO: Lógica de tamanho condicional
        pagerank = node.get('pagerank', 0)
        node_type = node.get('type', 'verbete')
        
        if node_type == 'category':
            # Multiplicador menor para categorias para que fiquem menores
            size = (15 + (pagerank * 50000)) 
        else: # É um verbete
            # Multiplicador original para verbetes
            size = (20 + (pagerank * 100000))

        # Prepara o nó para o Cytoscape
        cy_node = {
            'data': {
                'id': node_id,
                'label': node['titulo'],
                'type': node_type,
                'size': size,
                'color': node.get('color', node.get('community_color', '#6A737D')),
                'shape': node.get('shape', 'ellipse')
            },
            'position': node.get('position')
        }
        
        if node_type == 'verbete':
            cy_node['data']['community_id'] = node.get('community_id', -1)
            clean_categories = [c.replace('Categoria:', '').strip() for c in node.get('categorias', []) if c]
            cy_node['data']['categories_str'] = '|' + '|'.join(clean_categories) + '|'
            
            verbetes_data[node_id] = node
            
            cid = node.get('community_id', -1)
            if cid not in community_map:
                community_map[cid] = {'id': cid, 'color': node.get('community_color', '#6A737D'), 'size': 0}
            community_map[cid]['size'] += 1

        nodes_for_cy.append(cy_node)

        if node_type == 'category':
            all_categories.add(node['titulo'])
            verbetes_data[node_id] = node

    # Reconstrói a lista de arestas
    edges = []
    ids_renderizados = {node['data']['id'] for node in nodes_for_cy}

    for node in nodes_to_render:
        if node.get('type') == 'verbete':
            source_id = str(node['id'])
            
            # Arestas verbete -> verbete (hiperlinks)
            for ref_titulo in node.get('referencias', []):
                if ref_titulo in titulo_to_id:
                    target_id = str(titulo_to_id[ref_titulo])
                    # Garante que ambos os nós da aresta estão sendo renderizados
                    if source_id in ids_renderizados and target_id in ids_renderizados:
                        edges.append({'data': {'source': source_id, 'target': target_id}})
            
            # Arestas verbete -> categoria
            for cat_nome in node.get('categorias', []):
                target_id = f"cat_{cat_nome}"
                if source_id in ids_renderizados and target_id in ids_renderizados:
                    edges.append({'data': {'source': source_id, 'target': target_id}})

    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    verbetes_data_json = json.dumps(verbetes_data, ensure_ascii=False)
    nodes_json = json.dumps(nodes_for_cy, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    # ATUALIZAÇÃO NO JAVASCRIPT:
    # - Estilo condicional para 'shape' e 'background-color'
    # - Painel de detalhes com lógica if/else para 'verbete' ou 'category'
    js_template = """
        const nodeData = {verbetes_data_json};
        const communityData = {community_data_json};
        const categoryData = {categories_json};

        // Funções auxiliares (mesmas de antes)...
        function formatISODate(isoString) {
            if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
            try { return new Date(isoString).toLocaleString('pt-BR', { dateStyle: 'short', timeStyle: 'short' }); } catch (e) { return 'Data inválida'; }
        }
        function formatNumber(num) {
            if (typeof num !== 'number') return 'N/A';
            return num.toFixed(6);
        }
        function createTagList(dataArray, prefix = '') {
            if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
            return dataArray.map(item => `<span class="tag">${prefix}${item.trim()}</span>`).join('');
        }
        function createObjectTagList(dataObject) {
            if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
            return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${key}: ${value}</span>`).join('');
        }

        const layoutOptions = { name: 'preset', padding: 50, fit: true };

        const cy = cytoscape({
          container: document.getElementById('cy'),
          elements: { nodes: {nodes_json}, edges: {edges_json} },
          style: [
            // Estilo padrão para todos os nós
            { selector: 'node', style: {
                'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                'color': '#000000', 'font-size': '12px', 'text-valign': 'center',
                'text-halign': 'center', 'text-wrap': 'wrap', 'text-max-width': '100px',
                'border-color': '#000000', 'border-width': '1px', 'display': 'element'
            }},
            // Estilo específico para VERBETES (círculos)
            { selector: 'node[type="verbete"]', style: {
                'shape': 'ellipse',
                'background-color': 'data(color)'
            }},
            // Estilo específico para CATEGORIAS (triângulos brancos)
            { selector: 'node[type="category"]', style: {
                'shape': 'triangle',
                'background-color': '#FFFFFF' // Cor branca fixa para categorias
            }},
            { selector: 'edge', style: {
                'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                'curve-style': 'bezier', 'display': 'element'
            }},
            { selector: 'node:selected', style: {
                'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                'text-outline-color': '#FFFF00', 'text-outline-width': 1
            }},
            { selector: '.faded', style: { 'opacity': 0.1, 'text-opacity': 0 } }
          ],
          layout: layoutOptions
        });

        // Lógica de filtros (mesma de antes)...
        const communityFilter = new Choices('#community-filter', { removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true });
        const categoryFilter = new Choices('#category-filter', { removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...' });
        
        function populateFilters() {
            const communityChoices = communityData.sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id).map(c => {
                const label = c.name ? `${c.name} (${c.size})` : `Comunidade ${c.id} (${c.size}`;
                return { value: c.id, label: `<span class="color-swatch" style="background-color:${c.color};"></span> ${label}` };
            });
            communityFilter.setChoices(communityChoices, 'value', 'label', false);

            const categoryChoices = categoryData.map(cat => ({ value: cat, label: cat }));
            categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
        }
        
        function applyFilters() {
            const selectedCommunities = communityFilter.getValue(true);
            const selectedCategories = categoryFilter.getValue(true);
            
            if (selectedCommunities.length === 0 && selectedCategories.length === 0) {
                cy.elements().style('display', 'element'); return;
            }
            let nodesToShow = cy.nodes();
            if (selectedCommunities.length > 0) {
                const communitySelector = selectedCommunities.map(id => `[community_id = ${id}]`).join(', ');
                nodesToShow = nodesToShow.filter(communitySelector);
            }
            if (selectedCategories.length > 0) {
                // O filtro de categoria ainda funciona pois adicionamos 'categories_str' aos nós de verbetes
                const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${cat}|"]`).join(', ');
                nodesToShow = nodesToShow.filter(categorySelector);
            }
            cy.elements().style('display', 'none');
            nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
        }

        document.getElementById('community-filter').addEventListener('change', applyFilters);
        document.getElementById('category-filter').addEventListener('change', applyFilters);
        document.getElementById('select-all-communities').addEventListener('click', () => { communityFilter.setValue(communityData.map(c => String(c.id))); applyFilters(); });
        document.getElementById('clear-all-filters').addEventListener('click', () => { communityFilter.removeActiveItems(); categoryFilter.removeActiveItems(); applyFilters(); });

        populateFilters();

        // Lógica do painel de detalhes ATUALIZADA
        cy.on('tap', 'node', function(evt) {
            const node = evt.target;
            const data = nodeData[node.id()];
            let detailsHTML = '';

            if (data.type === 'verbete') {
                detailsHTML = `
                    <h3>Informações do Verbete</h3>
                    <div class="info-block"><strong>Título:</strong> ${data.titulo} || 'N/A'}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${data.link}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${data.community_id !== -1 ? data.community_id : 'N/A'}</div>
                    <div class="info-block"><strong>Autor da Criação:</strong> ${data.autor_criacao || 'N/A'}</div>
                    <div class="info-block"><strong>Data de Criação:</strong> ${formatISODate(data.data_criacao)}</div>
                    <div class="info-block"><strong>Última Edição:</strong> ${formatISODate(data.data_ultima_edicao)}</div>
                    <div class="info-block"><strong>Quantidade de Edições:</strong> ${data.quantidade_edicoes || 0}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${formatNumber(data.pagerank)}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${formatNumber(data.betweenness_centrality)}</div>
                    <div class="info-block"><strong>Closeness Centrality:</strong> ${formatNumber(data.closeness_centrality)}</div>
                    <div class="info-block"><strong>Degree Centrality:</strong> ${formatNumber(data.degree_centrality)}</div>
                    <hr>
                    <h3>Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${createTagList(data.categorias, 'Categoria: ')}</div>
                    <div class="info-block"><strong>Referências:</strong><br>${createTagList(data.referencias)}</div>
                `;
            } else { // Se for uma categoria
                detailsHTML = `
                    <h3>Informações da Categoria</h3>
                    <div class="info-block"><strong>Nome:</strong> ${data.titulo || 'N/A'}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${formatNumber(data.pagerank)}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${formatNumber(data.betweenness_centrality)}</div>
                    <div class="info-block"><strong>Closeness Centrality:</strong> ${formatNumber(data.closeness_centrality)}</div>
                    <div class="info-block"><strong>Degree Centrality (verbetes na categoria):</strong> ${formatNumber(data.degree_centrality)}</div>
                `;
            }
            
            document.getElementById('details').innerHTML = detailsHTML;
            cy.elements().addClass('faded');
            node.neighborhood().union(node).removeClass('faded');
        });
        
        // Outros listeners (mesmos de antes)...
        cy.on('tap', function(evt){ if (evt.target === cy) { cy.elements().removeClass('faded'); document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes"; } });
        const infoButton = document.getElementById('info-button');
        const modalOverlay = document.getElementById('modal-overlay');
        const modalClose = document.getElementById('modal-close');
        const layoutPropsEl = document.getElementById('layout-props');
        layoutPropsEl.textContent = JSON.stringify(layoutOptions, null, 2);
        infoButton.addEventListener('click', () => modalOverlay.classList.remove('hidden'));
        modalClose.addEventListener('click', () => modalOverlay.classList.add('hidden'));
        modalOverlay.addEventListener('click', (e) => { if (e.target === modalOverlay) modalOverlay.classList.add('hidden'); });
        const nodeColorPicker = document.getElementById('node-color-picker');
        const edgeColorPicker = document.getElementById('edge-color-picker');
        const edgeWidthSlider = document.getElementById('edge-width-slider');
        nodeColorPicker.addEventListener('input', e => cy.style().selector('node').style({ 'border-color': e.target.value }).update());
        edgeColorPicker.addEventListener('input', e => cy.style().selector('edge').style({ 'line-color': e.target.value }).update());
        edgeWidthSlider.addEventListener('input', e => cy.style().selector('edge').style({ 'width': e.target.value }).update());
    """

    js_code = js_template.replace('{verbetes_data_json}', verbetes_data_json) \
                         .replace('{nodes_json}', nodes_json) \
                         .replace('{edges_json}', edges_json) \
                         .replace('{community_data_json}', community_data_json) \
                         .replace('{categories_json}', categories_json)
    
    # Template HTML (mesmo de antes)
    html_template = f"""
        <!DOCTYPE html><html><head>
        <meta charset="utf-8"><title>Grafo Heterogêneo - Wikifavelas</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .control-item {{ display: flex; justify-content: space-between; align-items: center; margin-bottom: 8px; }}
            .control-item label {{ font-size: 14px; color: #c9d1d9; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            a:hover {{ text-decoration: underline; }}
            #info-button {{ position: absolute; top: 15px; right: 15px; z-index: 1000; background-color: rgba(255, 255, 255, 0.1); color: #c9d1d9; border: 1px solid #30363d; border-radius: 50%; width: 30px; height: 30px; font-size: 18px; font-weight: bold; cursor: pointer; }}
            .hidden, #modal-overlay.hidden {{ display: none; }}
            #modal-overlay {{ position: fixed; top: 0; left: 0; width: 100%; height: 100%; background-color: rgba(0, 0, 0, 0.7); z-index: 2000; display: flex; justify-content: center; align-items: center; }}
            #modal-content {{ background-color: #161b22; padding: 25px; border-radius: 8px; border: 1px solid #30363d; width: 90%; max-width: 600px; max-height: 80vh; overflow-y: auto; position: relative; }}
            #modal-close {{ position: absolute; top: 10px; right: 15px; font-size: 28px; font-weight: bold; color: #8b949e; cursor: pointer; }}
            #layout-props {{ background-color: #0d1117; padding: 15px; border-radius: 6px; white-space: pre-wrap; font-family: monospace; font-size: 13px; }}
            .choices {{ margin-bottom: 15px; }} .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d; padding: 2px 7.5px; min-height: 36px;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border: 1px solid #30363d; font-size: 12px; }}
            .choices__list--dropdown, .choices__list[aria-expanded] {{ background-color: #FFFFFF; border: 1px solid #30363d;}}
            .choices__list--dropdown .choices__item--selectable {{ color: #000000;}}
            .choices__list--dropdown .choices__item--selectable.is-highlighted {{ background-color: #000000; color: #FFFFFF;}}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head><body>
        <div id="cy"><button id="info-button">?</button></div>
        <div id="sidebar">
            <h2>Detalhes do Nó</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div><label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades (de Verbetes):</label><select id="community-filter" multiple></select></div>
                <div><label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias (de Verbetes):</label><select id="category-filter" multiple></select></div>
                <div class="filter-actions"><button id="select-all-communities" class="filter-button">Selecionar Todas</button><button id="clear-all-filters" class="filter-button">Limpar Filtros</button></div>
            </div>
            <div class="controls-container">
                <h3>Controles de Estilo</h3>
                <div class="control-item"><label for="node-color-picker">Cor da Borda do Nó:</label><input type="color" id="node-color-picker" value="#000000"></div>
                <div class="control-item"><label for="edge-color-picker">Cor da Aresta:</label><input type="color" id="edge-color-picker" value="#ffffff"></div>
                <div class="control-item"><label for="edge-width-slider">Largura da Aresta:</label><input type="range" id="edge-width-slider" min="0.5" max="10" step="0.5" value="1.5"></div>
            </div>
        </div>
        <div id="modal-overlay" class="hidden">
            <div id="modal-content">
                <span id="modal-close">&times;</span><h2>Propriedades do Layout</h2>
                <p>O grafo é renderizado usando as posições pré-calculadas no arquivo JSON.</p>
                <pre id="layout-props"></pre>
            </div>
        </div>
        <script>{js_code}</script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = os.path.join(output_dir, f'grafo_heterogeneo_{ts}.html')
    
    print(f"Salvando visualização final em '{output_html_path}'...")
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso!")

if __name__ == '__main__':
    gerar_visualizacao_heterogenea()

Carregando dados da rede heterogênea de 'dadosWikifavelas20250511\dados_rede_heterogenea.json'...
Preparando dados de nós, arestas e filtros para o Cytoscape...
Renderizando 3295 de 4607 nós (filtrando verbetes isolados).
Salvando visualização final em 'Visualizações\grafo_heterogeneo_20250811_172757.html'...
✅ Visualização gerada com sucesso!


# Visualização Verb <-> Verb (Hiperlink)

In [ ]:
# gerar_visualizacao_final.py
#
# Este script é a etapa final do pipeline, responsável por gerar a
# visualização interativa da rede.
#
# Etapas:
# 1. Carrega o arquivo JSON final ('dados_com_constraint.json'), que já
#    contém todas as métricas de rede e as coordenadas (x, y) dos nós.
# 2. Prepara os dados de nós e arestas para a biblioteca Cytoscape.js.
# 3. Utiliza o layout 'preset' do Cytoscape para posicionar os nós
#    exatamente de acordo com as coordenadas pré-calculadas.
# 4. Gera um arquivo HTML autocontido com o grafo interativo, filtros
#    de comunidade/categoria e um painel de detalhes.

import json
import os
from datetime import datetime
from pathlib import Path

def gerar_visualizacao_final():
    # Gera o arquivo HTML da visualização a partir dos dados finais pré-processados.

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    # Lê o arquivo final gerado pelo script 'analise_e_layout.py'
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados de nós, arestas e filtros para o Cytoscape...")
    
    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        # Fórmula para o tamanho do nó, baseada no PageRank
        size = (20 + (verb.get('pagerank', 0) * 60000))

        # Coleta todas as categorias únicas para o menu de filtro
        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': str(verb['id']),
                'label': verb['titulo'],
                'size': size,
                'color': verb.get('community_color', '#6A737D'),
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            # A posição já vem pronta do arquivo de entrada
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    # Prepara dados para o filtro de comunidades
    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    js_template = """
        const verbetes = {id_to_verbete_json};
        const communityData = {community_data_json};
        const categoryData = {categories_json};

        // Funções auxiliares para formatar os dados no painel de detalhes
        function formatISODate(isoString) {
            if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
            try { return new Date(isoString).toLocaleString('pt-BR', { dateStyle: 'short', timeStyle: 'short' }); } catch (e) { return 'Data inválida'; }
        }
        function formatNumber(num) {
            if (typeof num !== 'number') return 'N/A';
            return num.toFixed(6);
        }
        function createTagList(dataArray, prefix = '') {
            if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
            return dataArray.map(item => `<span class="tag">${prefix}${item.trim()}</span>`).join('');
        }
        function createObjectTagList(dataObject) {
            if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
            return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${key}: ${value}</span>`).join('');
        }

        const layoutOptions = {
            name: 'preset',
            padding: 50,
            fit: true,
        };

        const cy = cytoscape({
          container: document.getElementById('cy'),
          elements: { nodes: {nodes_json}, edges: {edges_json} },

          style: [
            { selector: 'node',
              style: {
                'label': 'data(label)',
                'width': 'data(size)',
                'height': 'data(size)',
                'background-color': 'data(color)',
                'color': '#000000', 
                'font-size': '12px', 
                'text-valign': 'center',
                'text-halign': 'center', 
                'text-wrap': 'wrap', 
                'text-max-width': '100px',
                'border-color': '#000000', 
                'border-width': '1px', 
                'display': 'element'
            }},
            
            { selector: 'edge',
              style: {
                'width': 1.5, 
                'line-color': '#ffffff', 
                'opacity': 0.3,
                'curve-style': 'bezier', 
                'display': 'element'
            }},
            
            { selector: 'node:selected',
              style: {
                'border-color': '#FFFF00', 
                'border-width': 6, 
                'color': '#000000',
                'text-outline-color': '#FFFF00', 
                'text-outline-width': 1
            }},

            { selector: '.faded', style: { 'opacity': 0.1, 'text-opacity': 0 } },
          ],
          layout: layoutOptions
        });

        const communityFilter = new Choices('#community-filter', {
            removeItemButton: true, 
            placeholder: true, 
            placeholderValue: 'Filtrar por comunidades...', 
            allowHTML: true
        });

        const categoryFilter = new Choices('#category-filter', {
            removeItemButton: true, 
            placeholder: true, 
            placeholderValue: 'Filtrar por categorias...'
        });

        function populateFilters() {
            const communityChoices = communityData
                .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                .map(c => {
                    const label = c.name ? `${c.name} (${c.size})` : `Comunidade ${c.id} (${c.size})`;
                    return { value: c.id, label: `<span class="color-swatch" style="background-color:${c.color};"></span> ${label}` };
                });
            communityFilter.setChoices(communityChoices, 'value', 'label', false);

            const categoryChoices = categoryData.map(cat => ({ value: cat, label: cat }));
            categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
        }

        function applyFilters() {
            const selectedCommunities = communityFilter.getValue(true);
            const selectedCategories = categoryFilter.getValue(true);

            if (selectedCommunities.length === 0 && selectedCategories.length === 0) {
                cy.elements().style('display', 'element');
                return;
            }

            let nodesToShow = cy.nodes();

            if (selectedCommunities.length > 0) {
                const communitySelector = selectedCommunities.map(id => `[community_id = ${id}]`).join(', ');
                nodesToShow = nodesToShow.filter(communitySelector);
            }

            if (selectedCategories.length > 0) {
                const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${cat}|"]`).join(', ');
                nodesToShow = nodesToShow.filter(categorySelector);
            }
            
            cy.elements().style('display', 'none');
            nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
        }

        document.getElementById('community-filter').addEventListener('change', applyFilters);
        document.getElementById('category-filter').addEventListener('change', applyFilters);

        populateFilters();

        document.getElementById('select-all-communities').addEventListener('click', () => {
            const allCommunityValues = communityData.map(c => String(c.id)); 
            communityFilter.setValue(allCommunityValues);
        });

        document.getElementById('clear-all-filters').addEventListener('click', () => {
            communityFilter.removeActiveItems();
            categoryFilter.removeActiveItems();
        });
        
        cy.on('tap', 'node', function(evt) {
            const node = evt.target;
            const data = verbetes[node.id()];
            
            let detailsHTML = `
                <h3>Informações Gerais</h3>
                <div class="info-block"><strong>Título:</strong> ${data.titulo || 'N/A'}</div>
                <div class="info-block"><strong>Link:</strong> <a href="${data.link}" target="_blank">Abrir na Wiki</a></div>
                <div class="info-block"><strong>ID da Comunidade:</strong> ${data.community_id !== -1 ? data.community_id : 'N/A'}</div>
                <div class="info-block"><strong>Autor da Criação:</strong> ${data.autor_criacao || 'N/A'}</div>
                <div class="info-block"><strong>Data de Criação:</strong> ${formatISODate(data.data_criacao)}</div>
                <div class="info-block"><strong>Última Edição:</strong> ${formatISODate(data.data_ultima_edicao)}</div>
                <div class="info-block"><strong>Quantidade de Edições:</strong> ${data.quantidade_edicoes || 0}</div>
                
                <hr>
                <h3>Métricas de Rede</h3>
                <div class="info-block"><strong>PageRank:</strong> ${formatNumber(data.pagerank)}</div>
                <div class="info-block"><strong>Betweenness Centrality:</strong> ${formatNumber(data.betweenness_centrality)}</div>
                <div class="info-block"><strong>Closeness Centrality:</strong> ${formatNumber(data.closeness_centrality)}</div>
                <div class="info-block"><strong>Clustering Coefficient:</strong> ${formatNumber(data.clustering_coefficient)}</div>
                <div class="info-block"><strong>Grau Total (conexões):</strong> ${data.total_degree || 0}</div>
                
                <hr>
                <h3>Dados de Conteúdo</h3>
                <div class="info-block">
                    <strong>Categorias:</strong><br>${createTagList(data.categorias, 'Categoria: ')}
                </div>
                <div class="info-block">
                    <strong>Usuários (edições):</strong><br>${createObjectTagList(data.usuarios_edicoes)}
                </div>
                <div class="info-block">
                    <strong>Referências (links):</strong><br>${createTagList(data.referencias)}
                </div>
            `;
            
            document.getElementById('details').innerHTML = detailsHTML;
            cy.elements().addClass('faded');
            node.neighborhood().union(node).removeClass('faded');
        });

        cy.on('tap', function(evt) {
          if (evt.target === cy) {
            cy.elements().removeClass('faded');
            document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
          }
        });
    """

    js_code = js_template.replace('{id_to_verbete_json}', id_to_verbete_json) \
                         .replace('{nodes_json}', nodes_json) \
                         .replace('{edges_json}', edges_json) \
                         .replace('{community_data_json}', community_data_json) \
                         .replace('{categories_json}', categories_json)

    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo de Referências - Wikifavelas</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{
                background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9;
                padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px;
            }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            a:hover {{ text-decoration: underline; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d; padding: 2px 7.5px; min-height: 36px;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border: 1px solid #30363d; font-size: 12px; }}
            .choices__list--dropdown, .choices__list[aria-expanded] {{ background-color: #FFFFFF; border: 1px solid #30363d;}}
            .choices__list--dropdown .choices__item--selectable {{ color: #000000;}}
            .choices__list--dropdown .choices__item--selectable.is-highlighted {{ background-color: #000000; color: #FFFFFF;}}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>

        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>{js_code}</script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_wikifavelas_final_{ts}.html'
    
    print(f"Salvando visualização final em '{output_html_path}'...")
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso!")

if __name__ == '__main__':
    gerar_visualizacao_final()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'...
Preparando dados de nós, arestas e filtros para o Cytoscape...
Salvando visualização final em 'Visualizações\grafo_wikifavelas_final_20250812_194933.html'...
✅ Visualização gerada com sucesso!


In [ ]:
# gerar_visualizacao_betweenness_com_filtros.py
#
# Versão que colore os nós do grafo com base na métrica de 'betweenness_centrality'
# em uma escala contínua, mas MANTÉM os filtros de comunidade e categoria.

import json
import os
from datetime import datetime
from pathlib import Path

# --- FUNÇÃO AUXILIAR PARA O MAPEAMENTO DE CORES ---
def _map_value_to_color(value, min_val, max_val, start_color=(0, 191, 255), end_color=(255, 255, 0)):
    """
    Mapeia um valor numérico para uma cor em um gradiente.
    start_color: azul claro (deepskyblue)
    end_color: amarelo
    """
    if max_val == min_val:
        return f'#{start_color[0]:02x}{start_color[1]:02x}{start_color[2]:02x}'
        
    # Normaliza o valor para o intervalo [0, 1]
    normalized_value = (value - min_val) / (max_val - min_val)
    
    # Interpola linearmente entre as cores de início e fim
    r = int(start_color[0] + normalized_value * (end_color[0] - start_color[0]))
    g = int(start_color[1] + normalized_value * (end_color[1] - start_color[1]))
    b = int(start_color[2] + normalized_value * (end_color[2] - start_color[2]))
    
    return f'#{r:02x}{g:02x}{b:02x}'

def gerar_visualizacao_final():
    """
    Gera o arquivo HTML da visualização a partir dos dados finais pré-processados.
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por Betweenness Centrality...")

    # Acha os valores min/max de betweenness para normalizar a cor
    betweenness_values = [v.get('betweenness_centrality', 0) for v in verbetes_enriquecidos]
    min_betweenness = min(betweenness_values)
    max_betweenness = max(betweenness_values)

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        
        # ATUALIZAÇÃO: Calcula a cor baseada na betweenness centrality
        betweenness = verb.get('betweenness_centrality', 0)
        node_color = _map_value_to_color(betweenness, min_betweenness, max_betweenness)

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': str(verb['id']),
                'label': verb['titulo'],
                'size': size,
                'color': node_color, # Usa a nova cor calculada
                'community_id': verb.get('community_id', -1), # Mantém o ID da comunidade para o filtro
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    # Prepara dados para o filtro de comunidades (mantido)
    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            # A cor aqui é apenas para a legenda do filtro, não para o grafo
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    # O template JS agora inclui novamente a lógica do filtro de comunidades
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Betweenness</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_betweenness_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_final()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'...
Preparando dados para o Cytoscape com coloração por Betweenness Centrality...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_betweenness_20250812_202614.html'!


In [ ]:
# gerar_visualizacao_closeness_percentil.py
#
# Versão que colore os nós do grafo com base em PERCENTIS da métrica 'closeness_centrality',
# usando uma escala de cores categórica para destacar os nós mais importantes.

import json
import os
from datetime import datetime
from pathlib import Path
import math

def gerar_visualizacao_por_closeness_percentil():
    """
    Gera o arquivo HTML da visualização com cores baseadas em percentis
    da centralidade de proximidade.
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por percentil de Closeness...")

    # --- ATUALIZAÇÃO: Lógica de Escala por Percentil ---
    # 1. Ordena os verbetes por closeness_centrality (do maior para o menor)
    verbetes_ordenados = sorted(
        verbetes_enriquecidos, 
        key=lambda v: v.get('closeness_centrality', 0), 
        reverse=True
    )
    
    # 2. Calcula os pontos de corte dos percentis
    total_verbetes = len(verbetes_ordenados)
    idx_top_1_percent = int(total_verbetes * 0.01)
    idx_top_10_percent = int(total_verbetes * 0.10)
    idx_top_50_percent = int(total_verbetes * 0.50)

    # 3. Cria um mapa de ID para cor com base na posição ordenada
    id_to_color = {}
    for i, verbete in enumerate(verbetes_ordenados):
        verbete_id = str(verbete['id'])
        if i < idx_top_1_percent:
            id_to_color[verbete_id] = '#FF0000'  # Vermelho
        elif i < idx_top_10_percent:
            id_to_color[verbete_id] = '#FFFF00'  # Amarelo
        elif i < idx_top_50_percent:
            id_to_color[verbete_id] = '#FFFFFF'  # Branco
        else:
            id_to_color[verbete_id] = '#00B1EC'  # Azul Claro

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        verb_id = str(verb['id'])
        
        # ATUALIZAÇÃO: Atribui a cor a partir do mapa de percentis
        node_color = id_to_color.get(verb_id, '#808080') # Cinza como fallback

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': verb_id,
                'label': verb['titulo'],
                'size': size,
                'color': node_color, # Usa a nova cor categórica
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Percentil de Closeness</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <div class="info-block"><strong>Closeness Centrality:</strong> ${{formatNumber(data.closeness_centrality)}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_closeness_percentil_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_por_closeness_percentil()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'...
Preparando dados para o Cytoscape com coloração por percentil de Closeness...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_closeness_percentil_20250812_205052.html'!


In [ ]:
# gerar_visualizacao_degree_percentil.py
#
# Versão que colore os nós do grafo com base em PERCENTIS da métrica 'total_degree',
# usando uma escala de cores categórica para destacar os nós mais conectados (hubs).

import json
import os
from datetime import datetime
from pathlib import Path
import math

def gerar_visualizacao_por_degree_percentil():
    """
    Gera o arquivo HTML da visualização com cores baseadas em percentis
    da centralidade de grau (total_degree).
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por percentil de Degree Centrality...")

    # --- ATUALIZAÇÃO: Lógica de Escala por Percentil para 'total_degree' ---
    # 1. Ordena os verbetes por total_degree (do maior para o menor)
    verbetes_ordenados = sorted(
        verbetes_enriquecidos, 
        key=lambda v: v.get('total_degree', 0), 
        reverse=True
    )
    
    # 2. Calcula os pontos de corte dos percentis
    total_verbetes = len(verbetes_ordenados)
    idx_top_1_percent = int(total_verbetes * 0.01)
    idx_top_10_percent = int(total_verbetes * 0.10)
    idx_top_50_percent = int(total_verbetes * 0.50)

    # 3. Cria um mapa de ID para cor com base na posição ordenada
    id_to_color = {}
    for i, verbete in enumerate(verbetes_ordenados):
        verbete_id = str(verbete['id'])
        if i < idx_top_1_percent:
            id_to_color[verbete_id] = '#FF0000'  # Vermelho
        elif i < idx_top_10_percent:
            id_to_color[verbete_id] = '#FFFF00'  # Amarelo
        elif i < idx_top_50_percent:
            id_to_color[verbete_id] = '#FFFFFF'  # Branco
        else:
            id_to_color[verbete_id] = '#00B1EC'  # Azul Claro

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        verb_id = str(verb['id'])
        
        # ATUALIZAÇÃO: Atribui a cor a partir do mapa de percentis
        node_color = id_to_color.get(verb_id, '#808080') # Cinza como fallback

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': verb_id,
                'label': verb['titulo'],
                'size': size,
                'color': node_color, # Usa a nova cor categórica
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Percentil de Degree</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <div class="info-block"><strong>Closeness Centrality:</strong> ${{formatNumber(data.closeness_centrality)}}</div>
                    <div class="info-block"><strong>Total Degree:</strong> ${{data.total_degree || 0}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_degree_percentil_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_por_degree_percentil()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'...
Preparando dados para o Cytoscape com coloração por percentil de Degree Centrality...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_degree_percentil_20250812_205102.html'!


In [ ]:
# gerar_visualizacao_edicoes_percentil.py
#
# Versão que colore os nós do grafo com base em PERCENTIS da métrica 'quantidade_edicoes',
# usando uma escala de cores categórica para destacar os verbetes mais ativos.

import json
import os
from datetime import datetime
from pathlib import Path
import math

def gerar_visualizacao_por_edicoes_percentil():
    """
    Gera o arquivo HTML da visualização com cores baseadas em percentis
    da quantidade de edições de cada verbete.
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por percentil de Quantidade de Edições...")

    # --- ATUALIZAÇÃO: Lógica de Escala por Percentil para 'quantidade_edicoes' ---
    # 1. Ordena os verbetes por quantidade_edicoes (do maior para o menor)
    verbetes_ordenados = sorted(
        verbetes_enriquecidos, 
        key=lambda v: v.get('quantidade_edicoes', 0), 
        reverse=True
    )
    
    # 2. Calcula os pontos de corte dos percentis
    total_verbetes = len(verbetes_ordenados)
    idx_top_1_percent = int(total_verbetes * 0.01)
    idx_top_10_percent = int(total_verbetes * 0.10)
    idx_top_50_percent = int(total_verbetes * 0.50)

    # 3. Cria um mapa de ID para cor com base na posição ordenada
    id_to_color = {}
    for i, verbete in enumerate(verbetes_ordenados):
        verbete_id = str(verbete['id'])
        if i < idx_top_1_percent:
            id_to_color[verbete_id] = '#FF0000'  # Vermelho
        elif i < idx_top_10_percent:
            id_to_color[verbete_id] = '#FFFF00'  # Amarelo
        elif i < idx_top_50_percent:
            id_to_color[verbete_id] = '#FFFFFF'  # Branco
        else:
            id_to_color[verbete_id] = "#00B1EC"  # Azul Claro

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        verb_id = str(verb['id'])
        
        # ATUALIZAÇÃO: Atribui a cor a partir do mapa de percentis
        node_color = id_to_color.get(verb_id, '#808080') # Cinza como fallback

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': verb_id,
                'label': verb['titulo'],
                'size': size,
                'color': node_color, # Usa a nova cor categórica
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Percentil de Edições</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <div class="info-block"><strong>Qtde. Edições:</strong> ${{data.quantidade_edicoes || 0}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness Centrality:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <div class="info-block"><strong>Closeness Centrality:</strong> ${{formatNumber(data.closeness_centrality)}}</div>
                    <div class="info-block"><strong>Total Degree:</strong> ${{data.total_degree || 0}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_edicoes_percentil_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_por_edicoes_percentil()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_posicoes_v3.json'...
Preparando dados para o Cytoscape com coloração por percentil de Quantidade de Edições...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_edicoes_percentil_20250812_205108.html'!


In [ ]:
# gerar_visualizacao_composta_percentil.py
#
# Versão final que colore os nós do grafo com base em PERCENTIS da
# métrica composta personalizada, destacando os "super-nós" da rede.

import json
import os
from datetime import datetime
from pathlib import Path
import math

def gerar_visualizacao_por_metrica_composta():
    """
    Gera o arquivo HTML da visualização com cores baseadas em percentis
    da métrica composta.
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    # Lê o arquivo gerado pelo script 'calcular_metrica_composta.py'
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por percentil da Métrica Composta...")

    # --- Lógica de Escala por Percentil para 'metrica_composta' ---
    # 1. Ordena os verbetes pela métrica composta (o arquivo já vem ordenado, mas garantimos aqui)
    verbetes_ordenados = sorted(
        verbetes_enriquecidos, 
        key=lambda v: v.get('metrica_composta', 0), 
        reverse=True
    )
    
    # 2. Calcula os pontos de corte dos percentis
    total_verbetes = len(verbetes_ordenados)
    idx_top_1_percent = int(total_verbetes * 0.01)
    idx_top_10_percent = int(total_verbetes * 0.10)
    idx_top_50_percent = int(total_verbetes * 0.50)

    # 3. Cria um mapa de ID para cor com base na posição ordenada
    id_to_color = {}
    for i, verbete in enumerate(verbetes_ordenados):
        verbete_id = str(verbete['id'])
        if i < idx_top_1_percent:
            id_to_color[verbete_id] = '#FF0000'  # Vermelho
        elif i < idx_top_10_percent:
            id_to_color[verbete_id] = '#FFFF00'  # Amarelo
        elif i < idx_top_50_percent:
            id_to_color[verbete_id] = '#FFFFFF'  # Branco
        else:
            id_to_color[verbete_id] = '#ADD8E6'  # Azul Claro

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        verb_id = str(verb['id'])
        
        # Atribui a cor a partir do mapa de percentis
        node_color = id_to_color.get(verb_id, '#808080') # Cinza como fallback

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': verb_id,
                'label': verb['titulo'],
                'size': size,
                'color': node_color,
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Métrica Composta</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>Métrica Composta:</strong> ${{formatNumber(data.metrica_composta)}}</div>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <div class="info-block"><strong>Closeness:</strong> ${{formatNumber(data.closeness_centrality)}}</div>
                    <div class="info-block"><strong>Total Degree:</strong> ${{data.total_degree || 0}}</div>
                    <div class="info-block"><strong>Qtde. Edições:</strong> ${{data.quantidade_edicoes || 0}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_metrica_composta_percentil_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_por_metrica_composta()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_metrica_composta.json'...
Preparando dados para o Cytoscape com coloração por percentil da Métrica Composta...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_metrica_composta_percentil_20250812_210643.html'!


In [ ]:
# gerar_visualizacao_constraint_percentil.py
#
# Versão que colore os nós do grafo com base em PERCENTIS da métrica 'constraint',
# usando uma escala de cores categórica para destacar os nós que atuam como
# pontes entre diferentes clusters (baixo constraint).

import json
import os
from datetime import datetime
from pathlib import Path
import math

def gerar_visualizacao_por_constraint_percentil():
    """
    Gera o arquivo HTML da visualização com cores baseadas em percentis
    da métrica de Restrição (Constraint) de Burt.
    """

    # --- 1. CONFIGURAÇÃO DE ARQUIVOS ---
    INPUT_DIR = Path('dados')
    # Lê o arquivo que já contém a métrica de constraint
    INPUT_FILENAME = 'dados_com_constraint.json'
    OUTPUT_DIR = Path('public')
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    input_path = INPUT_DIR / INPUT_FILENAME

    if not input_path.exists():
        print(f"ERRO: O arquivo de dados enriquecidos não foi encontrado em '{input_path}'")
        return

    # --- 2. CARREGAMENTO DOS DADOS ENRIQUECIDOS ---
    print(f"Carregando dados enriquecidos de '{input_path}'...")
    with open(input_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    verbetes_enriquecidos = dados.get('verbetes_completo', [])
    if not verbetes_enriquecidos:
        print("ERRO: Nenhum verbete encontrado no arquivo de entrada.")
        return
        
    id_to_verbete = {str(v['id']): v for v in verbetes_enriquecidos}
    titulos_ids = {v['titulo']: v['id'] for v in verbetes_enriquecidos}
    
    # --- 3. PREPARAÇÃO DOS DADOS PARA O CYTOSCAPE ---
    print("Preparando dados para o Cytoscape com coloração por percentil de Constraint...")

    # --- ATUALIZAÇÃO: Lógica de Escala por Percentil para 'constraint' ---
    # 1. Ordena os verbetes por constraint (do MENOR para o MAIOR, pois baixo é melhor)
    verbetes_ordenados = sorted(
        verbetes_enriquecidos, 
        key=lambda v: v.get('constraint', 1.0), # Default 1.0 (alto) se a métrica faltar
        reverse=False # Importante: ordem ascendente
    )
    
    # 2. Calcula os pontos de corte dos percentis
    total_verbetes = len(verbetes_ordenados)
    idx_top_1_percent = int(total_verbetes * 0.01)
    idx_top_10_percent = int(total_verbetes * 0.10)
    idx_top_50_percent = int(total_verbetes * 0.50)

    # 3. Cria um mapa de ID para cor com base na posição ordenada
    id_to_color = {}
    for i, verbete in enumerate(verbetes_ordenados):
        verbete_id = str(verbete['id'])
        if i < idx_top_1_percent:
            id_to_color[verbete_id] = '#FF0000'  # Vermelho (os melhores "brokers")
        elif i < idx_top_10_percent:
            id_to_color[verbete_id] = '#FFFF00'  # Amarelo
        elif i < idx_top_50_percent:
            id_to_color[verbete_id] = '#FFFFFF'  # Branco
        else:
            id_to_color[verbete_id] = '#00B1EC'  # Azul Claro (os mais "restritos")

    all_categories = set()
    nodes = []
    for verb in verbetes_enriquecidos:
        size = (20 + (verb.get('pagerank', 0) * 60000))
        verb_id = str(verb['id'])
        
        # Atribui a cor a partir do mapa de percentis
        node_color = id_to_color.get(verb_id, '#808080') # Cinza como fallback

        clean_cats = []
        if 'categorias' in verb and verb['categorias']:
            for cat in verb['categorias']:
                clean_cat = cat.replace('Categoria:', '').strip()
                if clean_cat:
                    all_categories.add(clean_cat)
                    clean_cats.append(clean_cat)

        nodes.append({
            'data': {
                'id': verb_id,
                'label': verb['titulo'],
                'size': size,
                'color': node_color,
                'community_id': verb.get('community_id', -1),
                'categories_str': '|' + '|'.join(clean_cats) + '|'
            },
            'position': verb.get('position')
        })

    edges = []
    for verb in verbetes_enriquecidos:
        source_vid = str(verb['id'])
        for ref_titulo in verb.get('referencias', []):
            if ref_titulo in titulos_ids:
                target_vid = str(titulos_ids[ref_titulo])
                edges.append({'data': {'source': source_vid, 'target': target_vid}})

    community_map = {}
    for verb in verbetes_enriquecidos:
        cid = verb.get('community_id', -1)
        if cid not in community_map:
            community_map[cid] = {'id': cid, 'color': verb.get('community_color', '#6A737D'), 'size': 0}
        community_map[cid]['size'] += 1
    
    if -1 in community_map:
        community_map[-1]['name'] = 'Isolados / Outros'
    
    community_data = list(community_map.values())
    sorted_categories = sorted(list(all_categories))

    # --- 4. GERAÇÃO DO CÓDIGO HTML E JAVASCRIPT ---
    id_to_verbete_json = json.dumps(id_to_verbete, ensure_ascii=False)
    nodes_json = json.dumps(nodes, ensure_ascii=False)
    edges_json = json.dumps(edges, ensure_ascii=False)
    community_data_json = json.dumps(community_data, ensure_ascii=False)
    categories_json = json.dumps(sorted_categories, ensure_ascii=False)
    
    html_template = f"""
        <!DOCTYPE html><html>
        <head>
        <meta charset="utf-8"><title>Grafo Wikifavelas - Cor por Percentil de Constraint</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/choices.js/public/assets/styles/choices.min.css"/>
        <script src="https://cdn.jsdelivr.net/npm/choices.js/public/assets/scripts/choices.min.js"></script>
        <script src="https://unpkg.com/cytoscape@3.24.0/dist/cytoscape.min.js"></script>
        <style>
            body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif; margin: 0; display: flex; height: 100vh; background-color: #0d1117; color: #c9d1d9; }}
            #cy {{ position: relative; width: 70%; height: 100%; background-color: #0d1117; }}
            #sidebar {{ width: 30%; padding: 20px; background-color: #161b22; overflow-y: auto; border-left: 1px solid #30363d; box-sizing: border-box; }}
            h2, h3 {{ color: #f0f6fc; border-bottom: 1px solid #30363d; padding-bottom: 10px; margin-top: 0; }}
            h3 {{ padding-top: 10px; padding-bottom: 8px; margin-bottom: 10px; font-size: 1.1em; }}
            hr {{ border: 0; height: 1px; background-color: #30363d; margin: 20px 0; }}
            .controls-container {{ padding-bottom: 15px; margin-bottom: 15px; border-bottom: 1px solid #30363d; }}
            .filter-actions {{ display: flex; justify-content: space-between; margin-top: 10px; }}
            .filter-button {{ background-color: #21262d; border: 1px solid #30363d; color: #c9d1d9; padding: 5px 10px; border-radius: 6px; cursor: pointer; font-size: 12px; }}
            .filter-button:hover {{ background-color: #30363d; }}
            .color-swatch {{ width: 12px; height: 12px; border: 1px solid #555; border-radius: 3px; margin-right: 8px; display: inline-block; vertical-align: middle; }}
            .info-block {{ margin-bottom: 8px; font-size: 14px; line-height: 1.5; }}
            .info-block strong {{ color: #8b949e; display: inline-block; width: 180px; vertical-align: top; }}
            .tag {{ display: inline-block; background-color: #21262d; color: #c9d1d9; padding: 4px 10px; margin: 2px; border-radius: 15px; font-size: 12px; border: 1px solid #30363d; }}
            .tag-empty {{ font-style: italic; color: #8b949e; }}
            a {{ color: #58a6ff; text-decoration: none; }}
            .choices {{ margin-bottom: 15px; }}
            .choices__inner {{ background-color: #0d1117; border-radius: 6px; border: 1px solid #30363d;}}
            .choices__list--multiple .choices__item {{ background-color: #0969da; border-color: #30363d; }}
            .choices__list--dropdown {{ background-color: #161b22; border-color: #30363d; }}
            .choices__placeholder {{ color: #8b949e; }}
        </style>
        </head>
        <body>
        <div id="cy"></div>
        <div id="sidebar">
            <h2>Detalhes do Verbete</h2>
            <div id="details">Clique em um nó para ver detalhes.</div>
            <div class="controls-container">
                <h3>Filtros</h3>
                <div>
                    <label for="community-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Comunidades:</label>
                    <select id="community-filter" multiple></select>
                </div>
                <div>
                    <label for="category-filter" style="font-size:14px; color:#8b949e; margin-bottom:5px; display:block;">Filtrar por Categorias:</label>
                    <select id="category-filter" multiple></select>
                </div>
                <div class="filter-actions">
                    <button id="select-all-communities" class="filter-button">Selecionar Todas</button>
                    <button id="clear-all-filters" class="filter-button">Limpar Filtros</button>
                </div>
            </div>
        </div>
        <script>
            const verbetes = {id_to_verbete_json};
            const communityData = {community_data_json};
            const categoryData = {categories_json};

            function formatISODate(isoString) {{
                if (!isoString || isoString.includes('Não encontrado') || isoString.includes('Erro')) return 'N/A';
                try {{ return new Date(isoString).toLocaleString('pt-BR', {{ dateStyle: 'short', timeStyle: 'short' }}); }} catch (e) {{ return 'Data inválida'; }}
            }}
            function formatNumber(num) {{
                if (typeof num !== 'number') return 'N/A';
                return num.toFixed(6);
            }}
            function createTagList(dataArray, prefix = '') {{
                if (!dataArray || dataArray.length === 0) return '<span class="tag-empty">Nenhuma</span>';
                return dataArray.map(item => `<span class="tag">${{prefix}}${{item.trim()}}</span>`).join('');
            }}
            function createObjectTagList(dataObject) {{
                if (!dataObject || Object.keys(dataObject).length === 0) return '<span class="tag-empty">Nenhum</span>';
                return Object.entries(dataObject).map(([key, value]) => `<span class="tag">${{key}}: ${{value}}</span>`).join('');
            }}

            const layoutOptions = {{ name: 'preset', padding: 50, fit: true }};

            const cy = cytoscape({{
              container: document.getElementById('cy'),
              elements: {{ nodes: {nodes_json}, edges: {edges_json} }},
              style: [
                {{ selector: 'node', style: {{
                    'label': 'data(label)', 'width': 'data(size)', 'height': 'data(size)',
                    'background-color': 'data(color)', 'color': '#000000', 
                    'font-size': '12px', 'text-valign': 'center', 'text-halign': 'center', 
                    'text-wrap': 'wrap', 'text-max-width': '100px',
                    'border-color': '#000000', 'border-width': '1px', 'display': 'element'
                }}}},
                {{ selector: 'edge', style: {{
                    'width': 1.5, 'line-color': '#ffffff', 'opacity': 0.3,
                    'curve-style': 'bezier', 'display': 'element'
                }}}},
                {{ selector: 'node:selected', style: {{
                    'border-color': '#FFFF00', 'border-width': 6, 'color': '#000000',
                    'text-outline-color': '#FFFF00', 'text-outline-width': 1
                }}}},
                {{ selector: '.faded', style: {{ 'opacity': 0.1, 'text-opacity': 0 }} }}
              ],
              layout: layoutOptions
            }});

            const communityFilter = new Choices('#community-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por comunidades...', allowHTML: true
            }});
            const categoryFilter = new Choices('#category-filter', {{
                removeItemButton: true, placeholder: true, placeholderValue: 'Filtrar por categorias...'
            }});

            function populateFilters() {{
                const communityChoices = communityData
                    .sort((a, b) => (a.id === -1) - (b.id === -1) || a.id - b.id)
                    .map(c => {{
                        const label = c.name ? `${{c.name}} (${{c.size}})` : `Comunidade ${{c.id}} (${{c.size}})`;
                        return {{ value: c.id, label: `<span class="color-swatch" style="background-color:${{c.color}};"></span> ${{label}}` }};
                    }});
                communityFilter.setChoices(communityChoices, 'value', 'label', false);

                const categoryChoices = categoryData.map(cat => ({{ value: cat, label: cat }}));
                categoryFilter.setChoices(categoryChoices, 'value', 'label', false);
            }}

            function applyFilters() {{
                const selectedCommunities = communityFilter.getValue(true);
                const selectedCategories = categoryFilter.getValue(true);

                if (selectedCommunities.length === 0 && selectedCategories.length === 0) {{
                    cy.elements().style('display', 'element');
                    return;
                }}
                let nodesToShow = cy.nodes();
                if (selectedCommunities.length > 0) {{
                    const communitySelector = selectedCommunities.map(id => `[community_id = ${{id}}]`).join(', ');
                    nodesToShow = nodesToShow.filter(communitySelector);
                }}
                if (selectedCategories.length > 0) {{
                    const categorySelector = selectedCategories.map(cat => `[categories_str *= "|${{cat}}|"]`).join(', ');
                    nodesToShow = nodesToShow.filter(categorySelector);
                }}
                cy.elements().style('display', 'none');
                nodesToShow.union(nodesToShow.connectedEdges()).style('display', 'element');
            }}

            document.getElementById('community-filter').addEventListener('change', applyFilters);
            document.getElementById('category-filter').addEventListener('change', applyFilters);
            populateFilters();

            document.getElementById('select-all-communities').addEventListener('click', () => {{
                const allCommunityValues = communityData.map(c => String(c.id)); 
                communityFilter.setValue(allCommunityValues);
            }});
            document.getElementById('clear-all-filters').addEventListener('click', () => {{
                communityFilter.removeActiveItems();
                categoryFilter.removeActiveItems();
            }});
            
            cy.on('tap', 'node', function(evt) {{
                const node = evt.target;
                const data = verbetes[node.id()];
                let detailsHTML = `
                    <h3>Informações Gerais</h3>
                    <div class="info-block"><strong>Título:</strong> ${{data.titulo || 'N/A'}}</div>
                    <div class="info-block"><strong>Link:</strong> <a href="${{data.link}}" target="_blank">Abrir na Wiki</a></div>
                    <div class="info-block"><strong>ID da Comunidade:</strong> ${{data.community_id !== -1 ? data.community_id : 'N/A'}}</div>
                    <hr>
                    <h3>Métricas de Rede</h3>
                    <div class="info-block"><strong>Constraint:</strong> ${{formatNumber(data.constraint)}}</div>
                    <div class="info-block"><strong>PageRank:</strong> ${{formatNumber(data.pagerank)}}</div>
                    <div class="info-block"><strong>Betweenness:</strong> ${{formatNumber(data.betweenness_centrality)}}</div>
                    <div class="info-block"><strong>Total Degree:</strong> ${{data.total_degree || 0}}</div>
                    <hr>
                    <h3>Dados de Conteúdo</h3>
                    <div class="info-block"><strong>Categorias:</strong><br>${{createTagList(data.categorias, 'Cat: ')}}</div>
                `;
                document.getElementById('details').innerHTML = detailsHTML;
                cy.elements().addClass('faded');
                node.neighborhood().union(node).removeClass('faded');
            }});

            cy.on('tap', function(evt) {{
              if (evt.target === cy) {{
                cy.elements().removeClass('faded');
                document.getElementById('details').innerHTML = "Clique em um nó para ver detalhes";
              }}
            }});
        </script>
        </body></html>
    """

    # --- 5. SALVANDO O ARQUIVO HTML FINAL ---
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_html_path = OUTPUT_DIR / f'grafo_constraint_percentil_{ts}.html'
    
    with open(output_html_path, 'w', encoding='utf-8') as f:
        f.write(html_template)

    print(f"✅ Visualização gerada com sucesso em '{output_html_path}'!")

if __name__ == '__main__':
    gerar_visualizacao_por_constraint_percentil()


Carregando dados enriquecidos de 'dadosWikifavelas20250511\dados_com_constraint.json'...
Preparando dados para o Cytoscape com coloração por percentil de Constraint...
✅ Visualização gerada com sucesso em 'Visualizações\grafo_constraint_percentil_20250814_011449.html'!
